-  retrieve data for the JAMA Nework paper on OD/OUD prevalence: modified on v7 by adding dx and meds as exposures

# 1. Import libraries

In [1]:
import sqlalchemy as sa
import json
import pandas as pd
import os
import urllib
import argparse
import csv
import pickle
import seaborn as sns
import copy

# 2. Connect to database

In [2]:
with open("config_HealthFacts.json") as f:
    config = json.load(f)

engine = sa.create_engine(config["vertica_connection_uri"])

connection = engine.connect()

# 3. Retrieve data records

## 3.1 Define concepts here

-  read the icd_code csv file

In [4]:
code_df = pd.read_csv(r'OpioidDx_ICD_Codes.csv')

In [5]:
code_df[code_df['icd_version']==9]

,icd_version,dx_group,dx_code,level,description
0,9,opioid poisoning,965.0,1,Poisoning by opiates and related narcotics
1,9,opioid poisoning,965.00,2,"Poisoning by opium (alkaloids), unspecified"
2,9,opioid poisoning,965.01,2,Poisoning by heroin
3,9,opioid poisoning,965.02,2,Poisoning by methadone
4,9,opioid poisoning,965.09,2,Poisoning by other opiates or related narcotics
5,9,opioid poisoning,970.1,1,Poisoning by opiate antagonists
6,9,opioid poisoning,E850.0,1,Accidental poisoning by heroin
7,9,opioid poisoning,E850.1,1,Accidental poisoning by methadone
8,9,opioid poisoning,E850.2,1,Accidental poisoning by other opiates or relat...
9,9,opioid adverse effect,E935.0,1,Heroin causing adverse effects in therapeutic use


In [6]:
code_df.loc[0, 'dx_code'] = '965.0'

In [7]:
code_df.loc[1, 'dx_code'] = '965.00'

-  opioid related dx code

In [8]:
opioids_poisoning_list = list(code_df[code_df['dx_group']=='opioid poisoning']['dx_code'])

In [9]:
opioids_adverse_effect_list =list(code_df[code_df['dx_group']=='opioid adverse effect']['dx_code'])

-  make OD and OUD lists

In [10]:
OD_list = opioids_adverse_effect_list + opioids_poisoning_list

In [11]:
OUD_list = opioid_abuse_list + opioids_dependence_list

-  opioid related dx string

In [12]:
OUD_list_string = "'305.50', '305.51', '305.52', '305.53', 'F11.10', 'F11.11', 'F11.120', 'F11.121', 'F11.122', 'F11.129', 'F11.14', 'F11.150', 'F11.151', 'F11.159', 'F11.181', 'F11.182', 'F11.188', 'F11.19', '304.0', '304.00', '304.01', '304.02', '304.03', '304.7', '304.70', '304.71', '304.72', '304.73', 'F11.20', 'F11.21', 'F11.220', 'F11.221', 'F11.222', 'F11.229', 'F11.23', 'F11.24', 'F11.250', 'F11.251', 'F11.259', 'F11.281', 'F11.282', 'F11.288', 'F11.29'"

In [13]:
# print(OD_list)

In [14]:
# new od codes
OD_list_string = "'E935.0', 'E935.1', 'E935.2', 'E940.1', 'T40.0X5', 'T40.0X5A', 'T40.0X5D', 'T40.0X5S', 'T40.2X5', 'T40.2X5A', 'T40.2X5D', 'T40.2X5S', 'T40.3X5', 'T40.3X5A', 'T40.3X5D', 'T40.3X5S', 'T40.4X5', 'T40.4X5A', 'T40.4X5D', 'T40.4X5S', 'T40.605', 'T40.605A', 'T40.605D', 'T40.605S', 'T40.695', 'T40.695A', 'T40.695D', 'T40.695S', '965.0', '965.00', '965.01', '965.02', '965.09', '970.1', 'E850.0', 'E850.1', 'E850.2', 'T40.0X1', 'T40.0X1A', 'T40.0X1D', 'T40.0X1S', 'T40.0X2', 'T40.0X2A', 'T40.0X2D', 'T40.0X2S', 'T40.0X3', 'T40.0X3A', 'T40.0X3D', 'T40.0X3S', 'T40.0X4', 'T40.0X4A', 'T40.0X4D', 'T40.0X4S', 'T40.1X1', 'T40.1X1A', 'T40.1X1D', 'T40.1X1S', 'T40.1X2', 'T40.1X2A', 'T40.1X2D', 'T40.1X2S', 'T40.1X3', 'T40.1X3A', 'T40.1X3D', 'T40.1X3S', 'T40.1X4', 'T40.1X4A', 'T40.1X4D', 'T40.1X4S', 'T40.2X1', 'T40.2X1A', 'T40.2X1D', 'T40.2X1S', 'T40.2X2', 'T40.2X2A', 'T40.2X2D', 'T40.2X2S', 'T40.2X3', 'T40.2X3A', 'T40.2X3D', 'T40.2X3S', 'T40.2X4', 'T40.2X4A', 'T40.2X4D', 'T40.2X4S', 'T40.3X1', 'T40.3X1A', 'T40.3X1D', 'T40.3X1S', 'T40.3X2', 'T40.3X2A', 'T40.3X2D', 'T40.3X2S', 'T40.3X3', 'T40.3X3A', 'T40.3X3D', 'T40.3X3S', 'T40.3X4', 'T40.3X4A', 'T40.3X4D', 'T40.3X4S', 'T40.4X1', 'T40.4X1A', 'T40.4X1D', 'T40.4X1S', 'T40.4X2', 'T40.4X2A', 'T40.4X2D', 'T40.4X2S', 'T40.4X3', 'T40.4X3A', 'T40.4X3D', 'T40.4X3S', 'T40.4X4', 'T40.4X4A', 'T40.4X4D', 'T40.4X4S', 'T40.601', 'T40.601A', 'T40.601D', 'T40.601S', 'T40.602', 'T40.602A', 'T40.602D', 'T40.602S', 'T40.603', 'T40.603A', 'T40.603D', 'T40.603S', 'T40.604', 'T40.604A', 'T40.604D', 'T40.604S', 'T40.691', 'T40.691A', 'T40.691D', 'T40.691S', 'T40.692', 'T40.692A', 'T40.692D', 'T40.692S', 'T40.693', 'T40.693A', 'T40.693D', 'T40.693S', 'T40.694', 'T40.694A', 'T40.694D', 'T40.694S'"

In [15]:
OD_OUD_list_string = OD_list_string + "," + OUD_list_string

-  define exposure keys, values and dicts 

In [16]:
exposure_keys = ['ur_status', 'census_region', 'hospital_scale', 'gender',\
                'marital_status', 'race', 'healthcare_setting', 'payment_type']

exposure_values = ['dh.urban_rural_status', 'dh.census_region', 'dh.bed_size_range', 'dp.gender',\
                  'dp.marital_status', 'dp.race', 'dpt.patient_type_desc', 'dpayer.payer_code_desc']

exposure_dict = dict(zip(exposure_keys, exposure_values))

-  define exposure values

In [17]:
years = range(2000,2018,1) # 2009-2015
census_regions = ["West", "Northeast", "Midwest", "South"]
ur_statuses = ["Urban", "Rural"]
genders = ["Male", "Female"]
care_settings = ["Inpatient", "Outpatient", "Emergency"]
hospital_scales = ["<5", "6-99", "100-199", "200-299", "300-499", "500+"]
marital_statuses = ["Divorced", "Married", "Single", "Widowed"]
payment_types = ["Medicaid", "Medicare", "Self-Pay"]
races = ["African American", "Asian", "Caucasian", "Hispanic", "Native American"]
age_cutoff = ">=0" 

-  define exposure values string

In [18]:
census_regions_string = "'West', 'Northeast', 'Midwest', 'South'"
ur_statuses_string = "'Urban', 'Rural'"
genders_string = "'Male', 'Female'" 
care_settings_string = "'Inpatient', 'Outpatient', 'Emergency'"
hospital_scales_string = "'<5', '6-99', '100-199', '200-299', '300-499', '500+'"
marital_statuses_string = "'Divorced', 'Married', 'Single', 'Widowed'"
payment_types_string = "'Medicaid', 'Medicare', 'Self-Pay'"
races_string = "'African American', 'Asian', 'Caucasian', 'Hispanic', 'Native American' "
age_cutoff = ">=0" 

- define exposure_list dict

In [19]:
exposure_list_values = [ur_statuses, census_regions, hospital_scales, genders,
                       marital_statuses, races, care_settings, payment_types]

exposure_list_dict = dict(zip(exposure_keys, exposure_list_values))

In [1]:
years = range(2009,2018,1) 

## 3.2 Get the number of OD and non-OD patients

In [28]:
# care_settings_string = "'Inpatient', 'Outpatient', 'Emergency'"
care_settings_string = "'Inpatient'"

-  OD: true

In [29]:
# define year
years = range(2009, 2018, 1)

for year in years:
    sql=""" SELECT COUNT(DISTINCT(temp.patient_sk)), COUNT(*) 
            FROM (
                SELECT dp.patient_sk, fe.age_in_years, dp.gender, dp.marital_status, dp.race,
                dh.census_region, dh.urban_rural_status, dh.bed_size_range,dpt.patient_type_desc, 
                dpayer.payer_code_desc, dm.medication_id
                FROM "HealthFacts".hf_d_patient dp
                INNER JOIN "HealthFacts".hf_f_encounter fe
                ON (dp.patient_id = fe.patient_id)
                INNER JOIN "HealthFacts".hf_d_hospital dh
                ON (fe.hospital_id = dh.hospital_id)
                INNER JOIN "HealthFacts".hf_d_patient_type dpt
                ON (fe.patient_type_id = dpt.patient_type_id)
                INNER JOIN "HealthFacts".hf_d_payer dpayer
                ON (fe.payer_id = dpayer.payer_id)
                INNER JOIN "HealthFacts".hf_f_diagnosis fd
                ON (fe.encounter_id = fd.encounter_id)
                INNER JOIN "HealthFacts".hf_d_diagnosis dd
                ON (fd.diagnosis_id = dd.diagnosis_id)
                INNER JOIN "HealthFacts".hf_f_medication fm
                ON (fe.encounter_id = fm.encounter_id)
                INNER JOIN "HealthFacts".hf_d_medication dm
                ON (fm.medication_id = dm.medication_id)
                INNER JOIN (
                    SELECT DISTINCT(dp.patient_sk)
                    FROM "HealthFacts".hf_d_patient dp
                    INNER JOIN "HealthFacts".hf_f_encounter fe
                    ON (dp.patient_id = fe.patient_id)
                    INNER JOIN "HealthFacts".hf_d_hospital dh
                    ON (fe.hospital_id = dh.hospital_id)
                    INNER JOIN "HealthFacts".hf_d_patient_type dpt
                    ON (fe.patient_type_id = dpt.patient_type_id)
                    INNER JOIN "HealthFacts".hf_d_payer dpayer
                    ON (fe.payer_id = dpayer.payer_id)
                    INNER JOIN "HealthFacts".hf_f_diagnosis fd
                    ON (fe.encounter_id = fd.encounter_id)
                    INNER JOIN "HealthFacts".hf_d_diagnosis dd
                    ON (fd.diagnosis_id = dd.diagnosis_id)
                    INNER JOIN "HealthFacts".hf_f_medication fm
                    ON (fe.encounter_id = fm.encounter_id)
                    INNER JOIN "HealthFacts".hf_d_medication dm
                    ON (fm.medication_id = dm.medication_id)
                    WHERE (
                    fe.age_in_years >= 0
                    AND fe.admitted_dt_tm >= '{year}-01-01' AND fe.admitted_dt_tm <= '{year}-12-31' 

                    AND dpt.patient_type_desc IN ({care_settings_string})

                    AND dd.diagnosis_code IN ({OD_list_string})
                    )
                )temp1
                ON dp.patient_sk = temp1.patient_sk
                WHERE (
                fe.age_in_years >= 0
                AND fe.admitted_dt_tm >= '{year}-01-01' AND fe.admitted_dt_tm <= '{year}-12-31' 

                AND dpt.patient_type_desc IN ({care_settings_string})

                )
            GROUP BY dp.patient_sk, admitted_dt_tm, age_in_years, gender, marital_status, race, census_region, 
            urban_rural_status, bed_size_range, patient_type_desc, payer_code_desc, dm.medication_id) temp
         """.format(year=year, 
                     census_regions_string=census_regions_string, ur_statuses_string=ur_statuses_string,
                    genders_string=genders_string, care_settings_string=care_settings_string, 
                    hospital_scales_string=hospital_scales_string, marital_statuses_string=marital_statuses_string, 
                    payment_types_string=payment_types_string, races_string=races_string,
                   OD_list_string=OD_list_string, OUD_list_string=OUD_list_string)

    result = pd.read_sql(sql,connection)
    df = pd.DataFrame(result)

    print(df)

   COUNT  COUNT
0   1477  56434
   COUNT  COUNT
0   1240  40334
   COUNT   COUNT
0   2749  126528
   COUNT   COUNT
0   4022  195748
   COUNT   COUNT
0   3844  131699
   COUNT   COUNT
0   7689  419735
   COUNT   COUNT
0   8313  445399
   COUNT   COUNT
0   8527  313412
   COUNT   COUNT
0   6783  234678


-  OD false

In [30]:
for year in years:
    sql=""" SELECT COUNT(DISTINCT(temp.patient_sk)), COUNT(*) 
            FROM (
                SELECT dp.patient_sk, fe.age_in_years, dp.gender, dp.marital_status, dp.race,
                dh.census_region, dh.urban_rural_status, dh.bed_size_range,dpt.patient_type_desc, 
                dpayer.payer_code_desc, dm.medication_id
                FROM "HealthFacts".hf_d_patient dp
                INNER JOIN "HealthFacts".hf_f_encounter fe
                ON (dp.patient_id = fe.patient_id)
                INNER JOIN "HealthFacts".hf_d_hospital dh
                ON (fe.hospital_id = dh.hospital_id)
                INNER JOIN "HealthFacts".hf_d_patient_type dpt
                ON (fe.patient_type_id = dpt.patient_type_id)
                INNER JOIN "HealthFacts".hf_d_payer dpayer
                ON (fe.payer_id = dpayer.payer_id)
                INNER JOIN "HealthFacts".hf_f_diagnosis fd
                ON (fe.encounter_id = fd.encounter_id)
                INNER JOIN "HealthFacts".hf_d_diagnosis dd
                ON (fd.diagnosis_id = dd.diagnosis_id)
                INNER JOIN "HealthFacts".hf_f_medication fm
                ON (fe.encounter_id = fm.encounter_id)
                INNER JOIN "HealthFacts".hf_d_medication dm
                ON (fm.medication_id = dm.medication_id)
                LEFT JOIN (
                    SELECT DISTINCT(dp.patient_sk)
                    FROM "HealthFacts".hf_d_patient dp
                    INNER JOIN "HealthFacts".hf_f_encounter fe
                    ON (dp.patient_id = fe.patient_id)
                    INNER JOIN "HealthFacts".hf_d_hospital dh
                    ON (fe.hospital_id = dh.hospital_id)
                    INNER JOIN "HealthFacts".hf_d_patient_type dpt
                    ON (fe.patient_type_id = dpt.patient_type_id)
                    INNER JOIN "HealthFacts".hf_d_payer dpayer
                    ON (fe.payer_id = dpayer.payer_id)
                    INNER JOIN "HealthFacts".hf_f_diagnosis fd
                    ON (fe.encounter_id = fd.encounter_id)
                    INNER JOIN "HealthFacts".hf_d_diagnosis dd
                    ON (fd.diagnosis_id = dd.diagnosis_id)
                    INNER JOIN "HealthFacts".hf_f_medication fm
                    ON (fe.encounter_id = fm.encounter_id)
                    INNER JOIN "HealthFacts".hf_d_medication dm
                    ON (fm.medication_id = dm.medication_id)
                    WHERE (
                    fe.age_in_years >= 0
                    AND fe.admitted_dt_tm >= '{year}-01-01' AND fe.admitted_dt_tm <= '{year}-12-31' 

                    AND dpt.patient_type_desc IN ({care_settings_string})

                    AND dd.diagnosis_code IN ({OD_list_string})
                    )
                )temp1
                ON dp.patient_sk = temp1.patient_sk
                WHERE (
                fe.age_in_years >= 0
                AND fe.admitted_dt_tm >= '{year}-01-01' AND fe.admitted_dt_tm <= '{year}-12-31' 

                AND dpt.patient_type_desc IN ({care_settings_string})

                AND temp1.patient_sk IS NULL
                )
            GROUP BY dp.patient_sk, admitted_dt_tm, age_in_years, gender, marital_status, race, census_region, 
            urban_rural_status, bed_size_range, patient_type_desc, payer_code_desc, dm.medication_id) temp
         """.format(year=year, 
                     census_regions_string=census_regions_string, ur_statuses_string=ur_statuses_string,
                    genders_string=genders_string, care_settings_string=care_settings_string, 
                    hospital_scales_string=hospital_scales_string, marital_statuses_string=marital_statuses_string, 
                    payment_types_string=payment_types_string, races_string=races_string,
                   OD_list_string=OD_list_string, OUD_list_string=OUD_list_string)

    result = pd.read_sql(sql,connection)
    df = pd.DataFrame(result)

    print(df)

    COUNT    COUNT
0  393563  8355801
    COUNT    COUNT
0  297077  5692413
    COUNT     COUNT
0  526862  12809795
    COUNT     COUNT
0  641482  16445365
    COUNT    COUNT
0  526676  9972373
     COUNT     COUNT
0  1032826  29398433
     COUNT     COUNT
0  1025993  28812681
    COUNT     COUNT
0  763376  15174664
    COUNT     COUNT
0  562370  10963827


## 3.3 Get exposure details for the study population

In [59]:
# re-connect to HF
with open("config_HealthFacts.json") as f:
    config = json.load(f)

engine = sa.create_engine(config["vertica_connection_uri"])

connection = engine.connect()

In [60]:
care_settings_string = "'Inpatient'"

In [ ]:
years = range(2009, 2018, 1)
# a for loop to process different years
for year in years:
    # retrieve data in batches 
    batchSize = 500000 # the maximum number of records in a retrieval batch
    numBatches = 1 # the number of batches for data retrieval in a year

    for i in range(numBatches):# note the difference between DISTINCT all vs DISTINCT patient_sk alone 
        # fe.admitted_dt_tm,
        sql=""" WITH ORDERED AS (
                SELECT dp.patient_sk, fe.encounter_id, fe.age_in_years, dp.gender, dp.marital_status, dp.race,
                dh.census_region, dh.urban_rural_status, dh.bed_size_range,dpt.patient_type_desc, 
                dpayer.payer_code_desc, dm.medication_id,
                ROW_NUMBER() OVER (ORDER BY dp.patient_sk) AS RowNumber

                FROM "HealthFacts".hf_d_patient dp
                INNER JOIN "HealthFacts".hf_f_encounter fe
                ON (dp.patient_id = fe.patient_id)
                INNER JOIN "HealthFacts".hf_d_hospital dh
                ON (fe.hospital_id = dh.hospital_id)
                INNER JOIN "HealthFacts".hf_d_patient_type dpt
                ON (fe.patient_type_id = dpt.patient_type_id)
                INNER JOIN "HealthFacts".hf_d_payer dpayer
                ON (fe.payer_id = dpayer.payer_id)
                INNER JOIN "HealthFacts".hf_f_diagnosis fd
                ON (fe.encounter_id = fd.encounter_id)
                INNER JOIN "HealthFacts".hf_d_diagnosis dd
                ON (fd.diagnosis_id = dd.diagnosis_id)
                INNER JOIN "HealthFacts".hf_f_medication fm
                ON (fe.encounter_id = fm.encounter_id)
                INNER JOIN "HealthFacts".hf_d_medication dm
                ON (fm.medication_id = dm.medication_id)
                INNER JOIN (
                    SELECT DISTINCT(dp.patient_sk)
                    FROM "HealthFacts".hf_d_patient dp
                    INNER JOIN "HealthFacts".hf_f_encounter fe
                    ON (dp.patient_id = fe.patient_id)
                    INNER JOIN "HealthFacts".hf_d_hospital dh
                    ON (fe.hospital_id = dh.hospital_id)
                    INNER JOIN "HealthFacts".hf_d_patient_type dpt
                    ON (fe.patient_type_id = dpt.patient_type_id)
                    INNER JOIN "HealthFacts".hf_d_payer dpayer
                    ON (fe.payer_id = dpayer.payer_id)
                    INNER JOIN "HealthFacts".hf_f_diagnosis fd
                    ON (fe.encounter_id = fd.encounter_id)
                    INNER JOIN "HealthFacts".hf_d_diagnosis dd
                    ON (fd.diagnosis_id = dd.diagnosis_id)
                    INNER JOIN "HealthFacts".hf_f_medication fm
                    ON (fe.encounter_id = fm.encounter_id)
                    INNER JOIN "HealthFacts".hf_d_medication dm
                    ON (fm.medication_id = dm.medication_id)
                    WHERE (
                    fe.age_in_years >= 0
                    AND fe.admitted_dt_tm >= '{year}-01-01' AND fe.admitted_dt_tm <= '{year}-12-31' 

                    AND dh.census_region IN ({census_regions_string})

                    AND dpt.patient_type_desc IN ({care_settings_string})

                    AND dd.diagnosis_code IN ({OD_list_string})
                    )
                )temp1
                ON dp.patient_sk = temp1.patient_sk
                WHERE (
                fe.age_in_years >= 0
                AND fe.admitted_dt_tm >= '{year}-01-01' AND fe.admitted_dt_tm <= '{year}-12-31' 

                AND dh.census_region IN ({census_regions_string})

                AND dpt.patient_type_desc IN ({care_settings_string})

                )
                GROUP BY dp.patient_sk, fe.encounter_id, admitted_dt_tm, age_in_years, gender, marital_status, race, census_region, 
                urban_rural_status, bed_size_range, patient_type_desc, payer_code_desc,dm.medication_id
                ORDER BY patient_sk, admitted_dt_tm)
                SELECT *
                FROM Ordered
                WHERE RowNumber BETWEEN {startRow} AND {endRow}
             """.format(year=year,startRow = 1 + i * batchSize, endRow = (i+1) * batchSize,
                         census_regions_string=census_regions_string, ur_statuses_string=ur_statuses_string,
                        genders_string=genders_string, care_settings_string=care_settings_string, 
                        hospital_scales_string=hospital_scales_string, marital_statuses_string=marital_statuses_string, 
                        payment_types_string=payment_types_string, races_string=races_string,
                       OD_list_string=OD_list_string, OUD_list_string=OUD_list_string)

        result = pd.read_sql(sql,connection)
        df = pd.DataFrame(result)
        print(len(set(list(df['patient_sk']))))
        df.to_csv(r'OpioidDx_Prevalence_Study/Datasets_20200604/{year}_dataset_OD_yes_batch_{numBatch}_df.csv'.format(year=year, numBatch=i+1), index=False)
        del df
      
    # when OD is false
    # retrieve data in batches 
    batchSize = 500000 
    numBatches_dict = {2009:17, 2010:12, 2011:26, 2012:53, 2013:20, 2014:60, 2015:58, 2016:31, 2017:22}
    numBatches = numBatches_dict.get(year)

    for i in range(numBatches):# note the difference between DISTINCT all vs DISTINCT patient_sk alone 
        # fe.admitted_dt_tm,
        sql=""" WITH ORDERED AS (
                SELECT dp.patient_sk, fe.encounter_id, fe.age_in_years, dp.gender, dp.marital_status, dp.race,
                dh.census_region, dh.urban_rural_status, dh.bed_size_range,dpt.patient_type_desc, 
                dpayer.payer_code_desc, dm.medication_id,
                ROW_NUMBER() OVER (ORDER BY dp.patient_sk) AS RowNumber

                FROM "HealthFacts".hf_d_patient dp
                INNER JOIN "HealthFacts".hf_f_encounter fe
                ON (dp.patient_id = fe.patient_id)
                INNER JOIN "HealthFacts".hf_d_hospital dh
                ON (fe.hospital_id = dh.hospital_id)
                INNER JOIN "HealthFacts".hf_d_patient_type dpt
                ON (fe.patient_type_id = dpt.patient_type_id)
                INNER JOIN "HealthFacts".hf_d_payer dpayer
                ON (fe.payer_id = dpayer.payer_id)
                INNER JOIN "HealthFacts".hf_f_diagnosis fd
                ON (fe.encounter_id = fd.encounter_id)
                INNER JOIN "HealthFacts".hf_d_diagnosis dd
                ON (fd.diagnosis_id = dd.diagnosis_id)
                INNER JOIN "HealthFacts".hf_f_medication fm
                ON (fe.encounter_id = fm.encounter_id)
                INNER JOIN "HealthFacts".hf_d_medication dm
                ON (fm.medication_id = dm.medication_id)
                LEFT JOIN (
                    SELECT DISTINCT(dp.patient_sk)
                    FROM "HealthFacts".hf_d_patient dp
                    INNER JOIN "HealthFacts".hf_f_encounter fe
                    ON (dp.patient_id = fe.patient_id)
                    INNER JOIN "HealthFacts".hf_d_hospital dh
                    ON (fe.hospital_id = dh.hospital_id)
                    INNER JOIN "HealthFacts".hf_d_patient_type dpt
                    ON (fe.patient_type_id = dpt.patient_type_id)
                    INNER JOIN "HealthFacts".hf_d_payer dpayer
                    ON (fe.payer_id = dpayer.payer_id)
                    INNER JOIN "HealthFacts".hf_f_diagnosis fd
                    ON (fe.encounter_id = fd.encounter_id)
                    INNER JOIN "HealthFacts".hf_d_diagnosis dd
                    ON (fd.diagnosis_id = dd.diagnosis_id)
                    INNER JOIN "HealthFacts".hf_f_medication fm
                    ON (fe.encounter_id = fm.encounter_id)
                    INNER JOIN "HealthFacts".hf_d_medication dm
                    ON (fm.medication_id = dm.medication_id)
                    WHERE (
                    fe.age_in_years >= 0
                    AND fe.admitted_dt_tm >= '{year}-01-01' AND fe.admitted_dt_tm <= '{year}-12-31' 

                    AND dh.census_region IN ({census_regions_string})

                    AND dpt.patient_type_desc IN ({care_settings_string})

                    AND dd.diagnosis_code IN ({OD_list_string})
                    )
                )temp1
                ON dp.patient_sk = temp1.patient_sk
                WHERE (
                fe.age_in_years >= 0
                AND fe.admitted_dt_tm >= '{year}-01-01' AND fe.admitted_dt_tm <= '{year}-12-31' 

                AND dpt.patient_type_desc IN ({care_settings_string})

                AND temp1.patient_sk IS NULL
                )
                GROUP BY dp.patient_sk, fe.encounter_id, admitted_dt_tm, age_in_years, gender, marital_status, race, census_region, 
                urban_rural_status, bed_size_range, patient_type_desc, payer_code_desc,dm.medication_id
                ORDER BY patient_sk, admitted_dt_tm)
                SELECT *
                FROM Ordered
                WHERE RowNumber BETWEEN {startRow} AND {endRow}
             """.format(year=year,startRow = 1 + i * batchSize, endRow = (i+1) * batchSize,
                         census_regions_string=census_regions_string, ur_statuses_string=ur_statuses_string,
                        genders_string=genders_string, care_settings_string=care_settings_string, 
                        hospital_scales_string=hospital_scales_string, marital_statuses_string=marital_statuses_string, 
                        payment_types_string=payment_types_string, races_string=races_string,
                       OD_list_string=OD_list_string, OUD_list_string=OUD_list_string)

        result = pd.read_sql(sql,connection)
        df = pd.DataFrame(result)
        df.to_csv(r'OpioidDx_Prevalence_Study/Datasets_20200604/{year}_dataset_OD_no_batch_{numBatch}_df.csv'.format(year=year, numBatch=i+1), index=False)
        del df